<a href="https://colab.research.google.com/github/avcinazlinazan/AdvancePython/blob/main/topic_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.chdir('/content/drive/My Drive/')

***Import Package and Dataset***

In [ ]:
datafile = '/content/drive/My Drive/Reviews.csv'

In [ ]:
!pip install pyldavis

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import pandas as pd
import numpy as np
#read the csv file
query_df = pd.read_csv(datafile,error_bad_lines=False)
query_df = query_df.iloc[:8000]#subset of dataframe
query_df['Review'] = query_df['Review'].astype(str)
query_df['Review'].head()

0                  3 yıldır kullanıyorum ve memnunum:)
1                        3 yıldır kullanıyorum müthiş 
2    Ürün bugün elime geçti çok fazla inceleme fırs...
3    Almaya karar verdim. Hemencecik geldi. Keyifle...
4    Günlük kullanımınızı çok çok iyi karsılıyor kı...
Name: Review, dtype: object

***Preprocess the text data***

In [ ]:
#text processing
import re
import string
from gensim import corpora, models, similarities



***`Remove Emoji`***

In [ ]:

#!/usr/bin/env python
import re

def deEmoji(text):

  emoji_pattern = re.compile("["
          u"\U0001F600-\U0001F64F"  # emoticons
          u"\U0001F300-\U0001F5FF"  # symbols & pictographs
          u"\U0001F680-\U0001F6FF"  # transport & map symbols
          u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                            "]+", flags=re.UNICODE)
  return str(emoji_pattern.sub('', text) )# no emoji


***Turkish and Arabic Stopwords***

In [ ]:
with open('stopwords-tr.txt', 'r') as f:
    myList = [line.strip() for line in f]

***Normalization Functions***

In [ ]:
def word_tokenize(sentence):
    """

    Args:
        sentence (str): any sentence.
    Returns:
        list: each item is a word.
    """


    acronym_each_dot = r"(?:[a-zğçşöüı]\.){2,}"
    acronym_end_dot = r"\b[a-zğçşöüı]{2,3}\."
    suffixes = r"[a-zğçşöüı]{3,}' ?[a-zğçşöüı]{0,3}"
    numbers = r"\d+[.,:\d]+"
    any_word = r"[a-zğçşöüı]+"
    punctuations = r"[a-zğçşöüı]*[.,!?;:]"
    word_regex = "|".join([acronym_each_dot,
                           acronym_end_dot,
                           suffixes,
                           numbers,
                           any_word,
                           punctuations])

    sentence = re.compile("%s"%word_regex, re.I).findall(sentence)
    return sentence

In [ ]:

def initial_clean(text):
     """
     Function to clean text-remove punctuations, lowercase text etc.
     """
     text = text.translate(str.maketrans('', '', string.punctuation))
     text = text.lower() # lower case text
     text = word_tokenize(text)
     return text


In [ ]:
def remove_stop_words(text):
     stop_words = myList
     return [word for word in text if word not in stop_words]

In [ ]:
def apply_all(text):
     """
     This function applies all the functions above into one
     """
     return remove_stop_words(initial_clean(deEmoji(text)))

In [ ]:
# clean texts and create new column "tokenized"
import time
t1 = time.time()
query_df['tokenized_texts'] = query_df['Review'].apply(apply_all) #noisy text data ->>>> normalized text data
t2 = time.time()
print("Time to clean and tokenize", len(query_df), "texts:", (t2-t1)/60, "min") #Time to clean and tokenize 3209 reviews: 0.21254388093948365 min

Time to clean and tokenize 8000 texts: 0.03089307149251302 min


In [ ]:
query_df.head(6)

,Rating,Review,tokenized_texts
0,1,3 yıldır kullanıyorum ve memnunum:),"[yıldır, kullanıyorum, memnunum]"
1,1,3 yıldır kullanıyorum müthiş,"[yıldır, kullanıyorum, müthiş]"
2,1,Ürün bugün elime geçti çok fazla inceleme fırs...,"[ürün, bugün, elime, geçti, fazla, inceleme, f..."
3,1,Almaya karar verdim. Hemencecik geldi. Keyifle...,"[almaya, karar, verdim, hemencecik, geldi, key..."
4,1,Günlük kullanımınızı çok çok iyi karsılıyor kı...,"[günlük, kullanımınızı, karsılıyor, mükemmel]"
5,1,gayet güzel,[güzel]


***Create Gensim dictionary and corpus***

In [ ]:
#LDA
import gensim
import pyLDAvis.gensim

In [ ]:
#Create a Gensim dictionary from the tokenized data
tokenized = query_df['tokenized_texts']
#Creating term dictionary of corpus, where each unique term is assigned an index.
dictionary = corpora.Dictionary(tokenized)
#Filter terms which occurs in less than 1 query and more than 80% of the queries.
dictionary.filter_extremes(no_below=1, no_above=0.8)
#convert the dictionary to a bag of words corpus
corpus = [dictionary.doc2bow(tokens) for tokens in tokenized]
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1)]]


In [ ]:
[[(dictionary[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('kullanıyorum', 1), ('memnunum', 1), ('yıldır', 1)]]

In [ ]:
#LDA
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 8, id2word=dictionary, passes=15)
ldamodel.save('mOdel.gensim')#save model
topics = ldamodel.print_topics(num_words=30)#words in each topic group


In [ ]:
#8 topic and topic groups
for topic in topics:
   print(topic)

(0, '0.024*"hızı" + 0.012*"usb" + 0.011*"yazma" + 0.009*"okuma" + 0.008*"ürün" + 0.007*"yeterli" + 0.007*"aldım" + 0.007*"veri" + 0.007*"aktarım" + 0.006*"sinyal" + 0.006*"mb" + 0.006*"gb" + 0.006*"30" + 0.005*"küçük" + 0.005*"tl" + 0.004*"yavaş" + 0.004*"bi" + 0.004*"güzel" + 0.004*"hız" + 0.004*"20" + 0.004*"normal" + 0.004*"ürünü" + 0.004*"kablosu" + 0.004*"mbs" + 0.004*"bellek" + 0.004*"dk" + 0.003*"bence" + 0.003*"16" + 0.003*"tercih" + 0.003*"hızlı"')
(1, '0.008*"ssd" + 0.006*"cok" + 0.004*"tesekkurler" + 0.004*"ses" + 0.004*"urun" + 0.004*"bilgisayar" + 0.003*"eski" + 0.003*"pc" + 0.003*"usb" + 0.003*"kablo" + 0.003*"yeni" + 0.003*"karşılıyor" + 0.002*"işletim" + 0.002*"hızlandı" + 0.002*"d" + 0.002*"veriyor" + 0.002*"idare" + 0.002*"eder" + 0.002*"degil" + 0.002*"bi" + 0.002*"guzel" + 0.002*"laptop" + 0.002*"icin" + 0.002*"yorumları" + 0.002*"kullaniyorum" + 0.002*"gözle" + 0.002*"bilgisayarımın" + 0.002*"icinde" + 0.002*"uyumlu" + 0.002*"sistemi"')
(2, '0.009*"aldım" + 0.008*"

In [ ]:
get_document_topics = ldamodel.get_document_topics(corpus[0])
print(get_document_topics)

[(0, 0.03126284), (1, 0.031250004), (2, 0.03126983), (3, 0.03126666), (4, 0.031250667), (5, 0.031258944), (6, 0.7811505), (7, 0.03129058)]


***Visualizing topics using pyLDAvis***

In [ ]:
#visualizing topics
lda_viz = gensim.models.ldamodel.LdaModel.load('mOdel.gensim')#load lda model
lda_display = pyLDAvis.gensim.prepare(lda_viz, corpus, dictionary, sort_topics=True)
pyLDAvis.display(lda_display)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


***Dominant Topic within documents***

In [ ]:
def dominant_topic(ldamodel,corpus,content):
     #Function to find the dominant topic in each query
     sent_topics_df = pd.DataFrame()
     # Get main topic in each query
     for i, row in enumerate(ldamodel[corpus]):
         row = sorted(row, key=lambda x: (x[1]), reverse=True)
         # Get the Dominant topic, Perc Contribution and Keywords for each query
         for j, (topic_num, prop_topic) in enumerate(row):
             if j == 0:  # =&gt; dominant topic
                 wp = ldamodel.show_topic(topic_num,topn=30)
                 topic_keywords = ", ".join([word for word, prop in wp])
                 sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
             else:
                 break
     sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']
     contents = pd.Series(content)#noisy data
     sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
     return(sent_topics_df)
df_dominant_topic = dominant_topic(ldamodel=ldamodel, corpus=corpus, content=query_df['Review'])
df_dominant_topic.head(10)

,Dominant_Topic,Perc_Contribution,Topic_Keywords,Review
0,6.0,0.7811,"ürün, tavsiye, ederim, güzel, elime, kullanışl...",3 yıldır kullanıyorum ve memnunum:)
1,6.0,0.7810,"ürün, tavsiye, ederim, güzel, elime, kullanışl...",3 yıldır kullanıyorum müthiş
2,3.0,0.8099,"hızlı, ürün, teşekkürler, kargo, cok, elime, h...",Ürün bugün elime geçti çok fazla inceleme fırs...
3,0.0,0.8905,"hızı, usb, yazma, okuma, ürün, yeterli, aldım,...",Almaya karar verdim. Hemencecik geldi. Keyifle...
4,2.0,0.4729,"aldım, 10, kurulumu, basit, kolay, sn, sorun, ...",Günlük kullanımınızı çok çok iyi karsılıyor kı...
5,6.0,0.5622,"ürün, tavsiye, ederim, güzel, elime, kullanışl...",gayet güzel
6,7.0,0.9125,"ürün, tavsiye, güzel, mouse, uygun, ederim, fi...",Çok kaliteli bir ürün ve fiyatı da uygun. Uzun...
7,7.0,0.9204,"ürün, tavsiye, güzel, mouse, uygun, ederim, fi...",yaklaşık 5 senedir kullanıyorum. defalarca düş...
8,7.0,0.6578,"ürün, tavsiye, güzel, mouse, uygun, ederim, fi...",Ürün günlük kullanım için çok uygun. Ürünle bi...
9,0.0,0.7771,"hızı, usb, yazma, okuma, ürün, yeterli, aldım,...",Ürünü bir arkadaşım 2009 yılında hediye olarak...


In [ ]:
df_dominant_topic.to_csv('outputFile.csv')#save your results